LOG WRITER DIMENSIONS

To look at full dataframe -> print(df) <br>
To look at seperated dataframe -> print(dfs)/print(dict['df_(dataframe number)']) <br>
To look at rank scores -> print(rank_score)

In [23]:
# WRITING THE LOG
import pandas as pd

# file_format = ['csv', 'avro', 'parquet', 'orc']
# schemas = ['st','vt', 'pt', 'extvt', 'wpt']
# partition = ['horizontal','predicate', 'subject']

print('Dimension Type:')
dimension = input()

file_format = []
schemas = []
partition = []

# take file format input
print("File Format: ")
while True:
    line = input()
    if line:
        file_format.append(line)
    else:
        break

# take schemas input
print("Schemas: ")
while True:
    line = input()
    if line:
        schemas.append(line)
    else:
        break

# take partition input
print("Partition: ")
while True:
    line = input()
    if line:
        partition.append(line)
    else:
        break

if dimension == 'storage': 
    idx = []
    li=[]
    # READ LOG - STORAGE FORMAT
    for i in schemas:
        for j in partition:
            for k in file_format:
                # check folder
                print(f'./logs/{k}/100M_{i}_{j}.txt')

                df = pd.read_csv(f'./logs/{k}/100M_{i}_{j}.txt',sep = ',', header = None)
                df = df.fillna(0)
                avg = df.mean(axis = 0)
                li.append(avg)
                idx.append(k + f"_{i}_{j}")

    # create dataframe
    df = pd.DataFrame(li, index = [idx])
    df = df.set_axis(["Q"+str(i+1) for i in range(11)], axis = 1)
    df = df.fillna(5000)
    # split dataframe
    dict = {}
    count = 0
    loop = len(file_format)
    dfs = []
    for i in range(int(len(partition) * len(file_format) * len(schemas) / len(file_format))):
        dict['df_{}'.format(i)] = df[count:loop]
        count = loop
        loop = loop+len(file_format)
        dfs.append(dict[f'df_{i}'])

    # CREATE RANK OCCURENCES
    import scipy.stats as ss
    import numpy as np

    rank_dataframe = []

    for x in dfs:
        df_ranks = x.T
        column_names = df_ranks.columns.to_numpy().tolist()
        column_names = [column_names for column_names, in column_names]
        
        df_ranks_occurences = []
        for index, row in df_ranks.iterrows():
            df_ranks_occurences.append(ss.rankdata(row, method = 'max'))

        df_ranks_occurences = pd.DataFrame(df_ranks_occurences)

        df_transpose = df_ranks_occurences.transpose()

        rank_table = []
        for index, row in df_transpose.iterrows():
            result_row = np.zeros(len(df_transpose.index))
            for i in range(len(row)):
                result_row[int(row[i])-1] +=1
            rank_table.append(result_row)

        rank_table = pd.DataFrame(rank_table)
        rank_table = rank_table.set_axis(column_names, axis = 'index')
        rank_table = rank_table.set_axis([i+1 for i in range(len(column_names))], axis='columns')

        # CREATE R SCORE
        q = 11
        d = len(rank_table.index)

        rank_score = []
        for index, row in rank_table.iterrows():
            s = 0
            for r in range(d):  
                s = s + (row[r+1]*(d-(r+1)) / (q*(d-1)) )
            rank_score.append(s)
        rank_score = pd.DataFrame(rank_score)
        rank_score = rank_score.set_axis(column_names, axis = 'index')
        rank_score = rank_score.set_axis(['Result'], axis='columns')
        rank_score = pd.concat([rank_table, rank_score], axis = 1)
        rank_dataframe.append(rank_score)
####################################################################################################
elif dimension == 'schema':
    idx = []
    li=[]
    # READ LOG - STORAGE FORMAT
    for j in partition:
        for k in file_format:
            for i in schemas:
                # check folder
                print(f'./logs/{k}/100M_{i}_{j}.txt')

                df = pd.read_csv(f'./logs/{k}/100M_{i}_{j}.txt',sep = ',', header = None)
                df = df.fillna(0)
                avg = df.mean(axis = 0)
                li.append(avg)
                idx.append(k + f"_{i}_{j}")

    # create dataframe
    df = pd.DataFrame(li, index = [idx])
    df = df.set_axis(["Q"+str(i+1) for i in range(11)], axis = 1)
    df = df.fillna(5000)
    # split dataframe
    dict = {}
    count = 0
    loop = len(schemas)
    dfs = []
    for i in range(int(len(partition) * len(file_format) * len(schemas) / len(schemas))):
        dict['df_{}'.format(i)] = df[count:loop]
        count = loop
        loop = loop+len(schemas)
        dfs.append(dict[f'df_{i}'])

    # CREATE RANK OCCURENCES
    import scipy.stats as ss
    import numpy as np

    rank_dataframe = []

    for x in dfs:
        df_ranks = x.T
        column_names = df_ranks.columns.to_numpy().tolist()
        column_names = [column_names for column_names, in column_names]
        
        df_ranks_occurences = []
        for index, row in df_ranks.iterrows():
            df_ranks_occurences.append(ss.rankdata(row, method = 'max'))

        df_ranks_occurences = pd.DataFrame(df_ranks_occurences)

        df_transpose = df_ranks_occurences.transpose()

        rank_table = []
        for index, row in df_transpose.iterrows():
            result_row = np.zeros(len(df_transpose.index))
            for i in range(len(row)):
                result_row[int(row[i])-1] +=1
            rank_table.append(result_row)

        rank_table = pd.DataFrame(rank_table)
        rank_table = rank_table.set_axis(column_names, axis = 'index')
        rank_table = rank_table.set_axis([i+1 for i in range(len(column_names))], axis='columns')

        # CREATE R SCORE
        q = 11
        d = len(rank_table.index)

        rank_score = []
        for index, row in rank_table.iterrows():
            s = 0
            for r in range(d):  
                s = s + (row[r+1]*(d-(r+1)) / (q*(d-1)) )
            rank_score.append(s)
        rank_score = pd.DataFrame(rank_score)
        rank_score = rank_score.set_axis(column_names, axis = 'index')
        rank_score = rank_score.set_axis(['Result'], axis='columns')
        rank_score = pd.concat([rank_table, rank_score], axis = 1)
        rank_dataframe.append(rank_score)
########################################################################################################
elif dimension == 'partition':
    idx = []
    li=[]
    # READ LOG - STORAGE FORMAT
    for i in schemas:
        for k in file_format:
            for j in partition:
                # check folder
                print(f'./logs/{k}/100M_{i}_{j}.txt')

                df = pd.read_csv(f'./logs/{k}/100M_{i}_{j}.txt',sep = ',', header = None)
                df = df.fillna(0)
                avg = df.mean(axis = 0)
                li.append(avg)
                idx.append(k + f"_{i}_{j}")

    # create dataframe
    df = pd.DataFrame(li, index = [idx])
    df = df.set_axis(["Q"+str(i+1) for i in range(11)], axis = 1)
    df = df.fillna(5000)
    # split dataframe
    dict = {}
    count = 0
    loop = len(partition)
    dfs = []
    for i in range(int(len(partition) * len(file_format) * len(schemas) / len(partition))):
        dict['df_{}'.format(i)] = df[count:loop]
        count = loop
        loop = loop+len(partition)
        dfs.append(dict[f'df_{i}'])

    # CREATE RANK OCCURENCES
    import scipy.stats as ss
    import numpy as np

    rank_dataframe = []

    for x in dfs:
        df_ranks = x.T
        column_names = df_ranks.columns.to_numpy().tolist()
        column_names = [column_names for column_names, in column_names]
        
        df_ranks_occurences = []
        for index, row in df_ranks.iterrows():
            df_ranks_occurences.append(ss.rankdata(row, method = 'max'))

        df_ranks_occurences = pd.DataFrame(df_ranks_occurences)

        df_transpose = df_ranks_occurences.transpose()

        rank_table = []
        for index, row in df_transpose.iterrows():
            result_row = np.zeros(len(df_transpose.index))
            for i in range(len(row)):
                result_row[int(row[i])-1] +=1
            rank_table.append(result_row)

        rank_table = pd.DataFrame(rank_table)
        rank_table = rank_table.set_axis(column_names, axis = 'index')
        rank_table = rank_table.set_axis([i+1 for i in range(len(column_names))], axis='columns')

        # CREATE R SCORE
        q = 11
        d = len(rank_table.index)

        rank_score = []
        for index, row in rank_table.iterrows():
            s = 0
            for r in range(d):  
                s = s + (row[r+1]*(d-(r+1)) / (q*(d-1)) )
            rank_score.append(s)
        rank_score = pd.DataFrame(rank_score)
        rank_score = rank_score.set_axis(column_names, axis = 'index')
        rank_score = rank_score.set_axis(['Result'], axis='columns')
        rank_score = pd.concat([rank_table, rank_score], axis = 1)
        rank_dataframe.append(rank_score)

else:
    print("choose 'storage' or 'partition' or 'schema' dimension")

Dimension Type:
File Format: 
Schemas: 
Partition: 
./logs/avro/100M_st_horizontal.txt
./logs/avro/100M_vt_horizontal.txt
./logs/avro/100M_pt_horizontal.txt
./logs/avro/100M_extvt_horizontal.txt
./logs/avro/100M_wpt_horizontal.txt
./logs/csv/100M_st_horizontal.txt
./logs/csv/100M_vt_horizontal.txt
./logs/csv/100M_pt_horizontal.txt
./logs/csv/100M_extvt_horizontal.txt
./logs/csv/100M_wpt_horizontal.txt
./logs/orc/100M_st_horizontal.txt
./logs/orc/100M_vt_horizontal.txt
./logs/orc/100M_pt_horizontal.txt
./logs/orc/100M_extvt_horizontal.txt
./logs/orc/100M_wpt_horizontal.txt
./logs/parquet/100M_st_horizontal.txt
./logs/parquet/100M_vt_horizontal.txt
./logs/parquet/100M_pt_horizontal.txt
./logs/parquet/100M_extvt_horizontal.txt
./logs/parquet/100M_wpt_horizontal.txt
./logs/avro/100M_st_subject.txt
./logs/avro/100M_vt_subject.txt
./logs/avro/100M_pt_subject.txt
./logs/avro/100M_extvt_subject.txt
./logs/avro/100M_wpt_subject.txt
./logs/csv/100M_st_subject.txt
./logs/csv/100M_vt_subject.txt
.

Save to excel for better view

In [11]:
# Create excel sheet
excel_table = dfs+rank_dataframe
xlwriter = pd.ExcelWriter('~/Desktop/rank_table.xlsx', engine='xlsxwriter')
row = 0
for dataframe in excel_table:
        dataframe.to_excel(xlwriter, sheet_name = 'schema_format_100M', startrow = row, startcol = 0)
        row = row + len(dataframe.index) + 2 
xlwriter.save()

CONFORMANCE PART

In [25]:
full_ranks_scores = pd.concat(rank_dataframe, axis = 0)
full_ranks_scores = full_ranks_scores['Result']
best_scores = full_ranks_scores.nlargest(5)

In [24]:
df_full_ranks = []
df = df.T
for index, row in df.iterrows():
    df_full_ranks.append(ss.rankdata(row, method = 'max'))

df_full_ranks = pd.DataFrame(df_full_ranks)
df_full_ranks = df_full_ranks.T
df_full_ranks = df_full_ranks.set_axis(df.columns, axis = 'index') #configurations
df_full_ranks = df_full_ranks.set_axis([i+1 for i in range(len(df.index))], axis='columns') #query

In [26]:
best_scores

avro_vt_predicate          0.772727
csv_vt_predicate           0.750000
parquet_extvt_predicate    0.750000
orc_extvt_predicate        0.727273
avro_pt_subject            0.704545
Name: Result, dtype: float64

In [27]:
best_scores = best_scores.index.to_numpy().tolist()
criteria_table = df_full_ranks.loc[best_scores]
print(criteria_table)

                         1   2   3   4   5   6   7   8   9   10  11
avro_vt_predicate        20  32  17  55  16  23  10  43  54  18   7
csv_vt_predicate          7  39  36  46  27  24   9  34  49  26  13
parquet_extvt_predicate  22  10  14  42  21  20  44  30  28  23  60
orc_extvt_predicate      30  13  11  43  28  28  54  27  18  17  60
avro_pt_subject          42  35   6  16  17  10  19  10  17  30  22


In [28]:
testing = criteria_table[criteria_table > 15]
testing

,1,2,3,4,5,6,7,8,9,10,11
avro_vt_predicate,20.0,32.0,17.0,55,16,23.0,NaN,43.0,54,18,NaN
csv_vt_predicate,NaN,39.0,36.0,46,27,24.0,NaN,34.0,49,26,NaN
parquet_extvt_predicate,22.0,NaN,NaN,42,21,20.0,44.0,30.0,28,23,60.0
orc_extvt_predicate,30.0,NaN,NaN,43,28,28.0,54.0,27.0,18,17,60.0
avro_pt_subject,42.0,35.0,NaN,16,17,NaN,19.0,NaN,17,30,22.0


In [29]:
testing1 = testing.count(axis=1)
testing1

avro_vt_predicate          9
csv_vt_predicate           8
parquet_extvt_predicate    9
orc_extvt_predicate        9
avro_pt_subject            8
dtype: int64

In [30]:
avg = testing1.mean(axis = 0)
avg

8.6

In [31]:
sum = testing1.sum(axis = 0)
sum

43

In [32]:
1 - (sum/55)

0.21818181818181814